In [1]:
import pandas as pd
import os, json 
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [3]:
df = pd.read_pickle("./problemsV2.pkl")
df.head()

,name,grade,IA1,IB1,IC1,ID1,IE1,IF1,IG1,IH1,...,FB18,FC18,FD18,FE18,FF18,FG18,FH18,FI18,FJ18,FK18
0,hhhjdkgpgndn,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,electric flash,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Captain patagonia,14,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,lemonwedge long var,14,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,Échauffement 4,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
x = df.drop(['grade', 'name'], axis=1) 
y = df.grade
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, stratify=y)

In [5]:
sm = SMOTE(random_state=27)
x_train_new, y_train_new = sm.fit_sample(x_train, y_train)
y_train_new.value_counts()

15    2175
14    2175
13    2175
12    2175
11    2175
10    2175
9     2175
21    2175
20    2175
19    2175
18    2175
17    2175
16    2175
Name: grade, dtype: int64

In [6]:
# train random forest on original data
clf = RandomForestClassifier(max_depth=20, n_estimators=500, criterion='entropy')
clf.fit(x_train, y_train)
# get accuracy of model 
# accuracy can be a misleading evaluation criteria but it will do as a starting metric
print("accuracy on training set " + str(clf.score(x_train, y_train)))
print("accuracy on testing set " + str(clf.score(x_test, y_test)))
# get relative importance of each feature (all features importance will add up to 1)
feat_importance = pd.DataFrame(zip(df.columns, clf.feature_importances_)).sort_values(1, ascending=False)
feat_importance.head(10)

accuracy on training set 0.911211014329868
accuracy on testing set 0.36217633562766305


,0,1
49,ID5,0.039260
138,IE13,0.028412
158,IC15,0.024481
106,IF10,0.024419
81,IC8,0.019079
389,FC18,0.017551
103,IC10,0.017280
83,IE8,0.015812
391,FE18,0.015492
17,IE2,0.015179


In [7]:
y_pred = clf.predict(x_test)
print(classification_report(y_test,y_pred, zero_division=0))

              precision    recall  f1-score   support

           9       0.43      0.95      0.59       933
          10       0.33      0.01      0.02       319
          11       0.24      0.04      0.07       422
          12       0.20      0.21      0.21       428
          13       0.23      0.25      0.24       352
          14       0.44      0.04      0.08       178
          15       0.23      0.04      0.07       187
          16       0.19      0.02      0.04       133
          17       0.00      0.00      0.00        53
          18       1.00      0.04      0.08        24
          19       0.00      0.00      0.00         8
          20       0.00      0.00      0.00         4
          21       0.00      0.00      0.00        10

    accuracy                           0.36      3051
   macro avg       0.25      0.12      0.11      3051
weighted avg       0.31      0.36      0.26      3051



In [8]:
# train random forest on smote data
clf_smote = RandomForestClassifier(max_depth=2, n_estimators=500, criterion='entropy')
clf_smote.fit(x_train_new, y_train_new)
print("accuracy on training set " + str(clf_smote.score(x_train, y_train)))
print("accuracy on testing set " + str(clf_smote.score(x_test, y_test)))
smote_feat_importance = pd.DataFrame(zip(df.columns, clf_smote.feature_importances_)).sort_values(1, ascending=False)
smote_feat_importance.head(10)

accuracy on training set 0.3290250070244451
accuracy on testing set 0.31465093411996065


,0,1
49,ID5,0.084397
106,IF10,0.055198
389,FC18,0.052650
138,IE13,0.051469
134,IA13,0.049917
391,FE18,0.046433
388,FB18,0.044885
158,IC15,0.044154
103,IC10,0.033167
81,IC8,0.032041


In [9]:
y_pred = clf_smote.predict(x_test)
print(classification_report(y_test,y_pred, zero_division=0))

              precision    recall  f1-score   support

           9       0.42      0.93      0.58       933
          10       0.12      0.02      0.03       319
          11       0.22      0.04      0.06       422
          12       0.18      0.03      0.05       428
          13       0.20      0.08      0.11       352
          14       0.10      0.03      0.04       178
          15       0.08      0.03      0.04       187
          16       0.10      0.07      0.08       133
          17       0.06      0.06      0.06        53
          18       0.04      0.21      0.06        24
          19       0.08      0.25      0.12         8
          20       0.01      0.25      0.01         4
          21       0.00      0.00      0.00        10

    accuracy                           0.31      3051
   macro avg       0.12      0.15      0.10      3051
weighted avg       0.24      0.31      0.22      3051

